## Vision Transformer model for training Food101 datasets
- number of parameters : 86,567,656

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
# Checking versions of libraries for google colab: torch, torchvision

try:
    import torch
    import torchvision
    print(f'torch version: {torch.__version__}')
    print(f'torchvision version: {torchvision.__version__}')
    assert int(torch.__version__.split('.')[1]) >= 12, 'torch version should be 1.12+'
    assert int(torchvision.__version__.split('.')[1]) >= 13, 'torchvision version should be 0.13+'
except:
    print(f'[INFO] torch/torchvision versions not as required, installing required versions.')


c:\Users\TimJimTangtong\Miniconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch version: 1.13.1
torchvision version: 0.14.1


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms, datasets

from torchmetrics import Accuracy

from pathlib import Path
from tqdm.auto import tqdm
from typing import List

from collections import Counter

import sys
sys.path.append('./drive/MyDrive/Colab Notebooks/Paper Replicating/')
sys.path.append('..')
from python_scripts import engine, data_setup, utils

c:\Users\TimJimTangtong\Miniconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CUR_FORDER = './drive/MyDrive/Colab Notebooks/Paper Replicating/'
BATCH_SIZE = 32

In [3]:
# Checking GPU

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [4]:
vitb16_weights = torchvision.models.ViT_B_16_Weights.DEFAULT
vitb16_food101_transforms = vitb16_weights.transforms()

In [ ]:
data_dir = Path('C:/Users/TimJimTangtong/Deep learning projects/DL_Paper_Replicating/data/extracted')

train_data_vitb16_food101 = datasets.Food101(
    root=data_dir,
    split='train',
    transform=vitb16_food101_transforms,
    download=True
)
test_data_vitb16_food101 = datasets.Food101(
    root=data_dir,
    split='test',
    transform=vitb16_food101_transforms,
    download=True
)

class_names_food101 = train_data_vitb16_food101.classes

In [5]:
dropout_p_list = [0]
learning_rate_list = [1e-4]
weight_decay_list = [0]
epochs_list = [30]
batch_size_list = [16]

In [ ]:
sub32_train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_vitb16_food101,
    split_size=1, # you can start with small number of train set to check normality at first
    seed=42
)
sub32_test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_vitb16_food101,
    split_size=1,
    seed=42
)

tuning_results = engine.vitb16_HP_tune_train_TSboard(
    model_weights=vitb16_weights,
    train_dataset=sub32_train_dataset,
    test_dataset=sub32_test_dataset,
    dropout_p_list=dropout_p_list,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_writer=True,
    device=device
)

[INFO] Splitting dataset of length 75750 into splits of size: 75750 and 0
[INFO] Splitting dataset of length 25250 into splits of size: 25250 and 0
[INFO] Creating SummaryWriter saving to ../runs\2023-03-01\vitb16_food101_test\ViT_b2_food101\DP_0_LR_0.0001_WD_0_EP_50_BS_32


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 0 | Train_loss: 2.6017, Train_acc: 0.4561 | Test_loss: 1.6942, Test_acc: 0.6132


c:\Users\TimJimTangtong\Miniconda3\envs\torch-gpu\lib\site-packages\torch\__init__.py:853: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message
  2%|▏         | 1/50 [25:25<20:45:31, 1525.14s/it]

Epoch: 1 | Train_loss: 1.6523, Train_acc: 0.6038 | Test_loss: 1.3581, Test_acc: 0.6662


...

 96%|█████████▌| 48/50 [20:51:52<52:23, 1571.57s/it]  

Epoch: 48 | Train_loss: 0.6697, Train_acc: 0.8275 | Test_loss: 0.8238, Test_acc: 0.7742


 98%|█████████▊| 49/50 [21:18:05<26:12, 1572.06s/it]

Epoch: 49 | Train_loss: 0.6660, Train_acc: 0.8287 | Test_loss: 0.8242, Test_acc: 0.7739


100%|██████████| 50/50 [21:44:20<00:00, 1565.21s/it]


In [ ]:
import pandas as pd
pd.DataFrame(tuning_results)

In [6]:
vitb16_food101_transforms_train_2 = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

vitb16_food101_transforms_test_2 = transforms.Compose([
    transforms.Resize(225),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [7]:
# when using colab: /content/extracted
# when using in window: C:/Users/TimJimTangtong/Deep learning projects/DL_Paper_Replicating/data/extracted
data_dir = Path('C:/Users/TimJimTangtong/Deep learning projects/DL_Paper_Replicating/data/extracted')

train_data_vitb16_food101 = datasets.Food101(
    root=data_dir,
    split='train',
    transform=vitb16_food101_transforms_train_2,
    download=True
)
test_data_vitb16_food101 = datasets.Food101(
    root=data_dir,
    split='test',
    transform=vitb16_food101_transforms_test_2,
    download=True
)

class_names_food101 = train_data_vitb16_food101.classes

In [ ]:
sub32_train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_vitb16_food101,
    split_size=1,
    seed=42
)
sub32_test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_vitb16_food101,
    split_size=1,
    seed=42
)

tuning_results = engine.vitb16_HP_tune_train_TSboard(
    model_weights=vitb16_weights,
    train_dataset=sub32_train_dataset,
    test_dataset=sub32_test_dataset,
    dropout_p_list=dropout_p_list,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_writer=True,
    device=device
)

[INFO] Splitting dataset of length 75750 into splits of size: 75750 and 0
[INFO] Splitting dataset of length 25250 into splits of size: 25250 and 0
[INFO] Creating SummaryWriter saving to ../runs\2023-03-02\vitb16_food101_test\ViT_b2_food101\DP_0_LR_0.0001_WD_0_EP_30_BS_32


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 0 | Train_loss: 3.2617, Train_acc: 0.3078 | Test_loss: 1.9494, Test_acc: 0.5669


c:\Users\TimJimTangtong\Miniconda3\envs\torch-gpu\lib\site-packages\torch\__init__.py:853: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message
  3%|▎         | 1/30 [27:48<13:26:15, 1668.13s/it]

Epoch: 1 | Train_loss: 2.4535, Train_acc: 0.4347 | Test_loss: 1.5630, Test_acc: 0.6251


...

 93%|█████████▎| 28/30 [12:43:39<54:08, 1624.08s/it]  

Epoch: 28 | Train_loss: 1.6884, Train_acc: 0.5854 | Test_loss: 0.9223, Test_acc: 0.7606


 97%|█████████▋| 29/30 [13:10:25<26:58, 1618.73s/it]

Epoch: 29 | Train_loss: 1.6850, Train_acc: 0.5861 | Test_loss: 0.9180, Test_acc: 0.7629


100%|██████████| 30/30 [13:36:41<00:00, 1633.38s/it]


In [ ]:
import pandas as pd
pd.DataFrame(tuning_results)

,dropout_p,learning_rate,weight_decay,epochs,batch_size,train_loss,test_loss,train_acc,test_acc
0,0,0.0001,0,30,32,1.6850339,0.918016,0.58610904,0.762856


In [17]:
torch.cuda.memory_summary()

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 2         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |    5397 MB |   24732 MB |   24732 MB |\n|       from large pool |       0 B  |    5391 MB |   24721 MB |   24721 MB |\n|       from small pool |       0 B  |       6 MB |      10 MB |      10 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |    5397 MB |   24732 MB |   24732 MB |\n|       from large pool |       0 B  |    5391 MB |

In [8]:
sub32_train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_vitb16_food101,
    split_size=1,
    seed=42
)
sub32_test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_vitb16_food101,
    split_size=1,
    seed=42
)

tuning_results = engine.vitb16_HP_tune_train_TSboard_nonfreeze(
    model_weights=vitb16_weights,
    train_dataset=sub32_train_dataset,
    test_dataset=sub32_test_dataset,
    dropout_p_list=dropout_p_list,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_writer=True,
    device=device
)

[INFO] Splitting dataset of length 75750 into splits of size: 75750 and 0
[INFO] Splitting dataset of length 25250 into splits of size: 25250 and 0
[INFO] Creating SummaryWriter saving to ../runs\2023-03-10\vitb16_food101_test\ViT_b2_food101\DP_0_LR_0.0001_WD_0_EP_30_BS_28


  3%|▎         | 1/30 [40:59<19:48:50, 2459.67s/it]

Epoch: 0 | Train_loss: 2.2224, Train_acc: 0.4672 | Test_loss: 0.9636, Test_acc: 0.7457


  7%|▋         | 2/30 [1:21:34<19:01:04, 2445.18s/it]

Epoch: 1 | Train_loss: 1.6013, Train_acc: 0.5965 | Test_loss: 0.8056, Test_acc: 0.7794


...

 90%|█████████ | 27/30 [18:41:26<2:05:15, 2505.18s/it]

Epoch: 26 | Train_loss: 0.6118, Train_acc: 0.8331 | Test_loss: 0.6414, Test_acc: 0.8339
Epoch: 27 | Train_loss: 0.6016, Train_acc: 0.8349 | Test_loss: 0.6487, Test_acc: 0.8323


 97%|█████████▋| 29/30 [20:05:02<41:46, 2506.18s/it]  

Epoch: 28 | Train_loss: 0.5801, Train_acc: 0.8408 | Test_loss: 0.6568, Test_acc: 0.8336
Epoch: 29 | Train_loss: 0.5737, Train_acc: 0.8432 | Test_loss: 0.6794, Test_acc: 0.8250


100%|██████████| 30/30 [20:46:45<00:00, 2493.53s/it]


In [9]:
import pandas as pd
pd.DataFrame(tuning_results)

,dropout_p,learning_rate,weight_decay,epochs,batch_size,train_loss,test_loss,train_acc,test_acc
0,0,0.0001,0,30,28,0.5737325,0.6793879,0.8432206,0.8250382


In [8]:
sub32_train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_vitb16_food101,
    split_size=1,
    seed=42
)
sub32_test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_vitb16_food101,
    split_size=1,
    seed=42
)

tuning_results = engine.vitb16_HP_tune_train_TSboard_nonfreeze_gradient_accumulation(
    model_weights=vitb16_weights,
    train_dataset=sub32_train_dataset,
    test_dataset=sub32_test_dataset,
    dropout_p_list=dropout_p_list,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_writer=True,
    device=device,
    accumulation_num=4
)

[INFO] Splitting dataset of length 75750 into splits of size: 75750 and 0
[INFO] Splitting dataset of length 25250 into splits of size: 25250 and 0
[INFO] Creating SummaryWriter saving to ../runs\2023-03-12\vitb16_food101_test\ViT_b2_food101\DP_0_LR_0.0001_WD_0_EP_30_BS_16


  3%|▎         | 1/30 [40:35<19:37:01, 2435.24s/it]

Epoch: 0 | Train_loss: 0.5249, Train_acc: 0.5015 | Test_loss: 0.8935, Test_acc: 0.7699


  7%|▋         | 2/30 [1:21:13<18:57:09, 2436.77s/it]

Epoch: 1 | Train_loss: 0.3646, Train_acc: 0.6298 | Test_loss: 0.7076, Test_acc: 0.8072
Epoch: 2 | Train_loss: 0.3233, Train_acc: 0.6685 | Test_loss: 0.6725, Test_acc: 0.8148


...

 93%|█████████▎| 28/30 [19:38:46<1:27:56, 2638.06s/it]

Epoch: 27 | Train_loss: 0.1304, Train_acc: 0.8587 | Test_loss: 0.6081, Test_acc: 0.8448


 97%|█████████▋| 29/30 [20:23:36<44:13, 2653.63s/it]  

Epoch: 28 | Train_loss: 0.1260, Train_acc: 0.8627 | Test_loss: 0.6481, Test_acc: 0.8373
Epoch: 29 | Train_loss: 0.1252, Train_acc: 0.8620 | Test_loss: 0.6393, Test_acc: 0.8391


100%|██████████| 30/30 [21:08:33<00:00, 2537.13s/it]


In [9]:
import pandas as pd
pd.DataFrame(tuning_results)

,dropout_p,learning_rate,weight_decay,epochs,batch_size,train_loss,test_loss,train_acc,test_acc
0,0,0.0001,0,30,16,0.12522165,0.63925093,0.86202484,0.8390991
